# CS 39006: Networks Lab test

## Name : Radhika Patwari
## Roll no. : 18CS10062

## Installing 1 library required for various operations

    xmltodict : It parses xml file (.pdml extension) into dictionary for carrying various operations [Info : https://pypi.org/project/xmltodict/]
   


In [2]:
!pip install xmltodict

  Using cached https://files.pythonhosted.org/packages/28/fd/30d5c1d3ac29ce229f6bdc40bbc20b28f716e8b363140c26eff19122d8a5/xmltodict-0.12.0-py2.py3-none-any.whl


## Importing the required libraries

In [3]:
import xmltodict                                            # xml to dict library
import json                                                 # printing and checking in json format

In [4]:
xml_file = '/content/filtered_smtp.pdml'
with open(xml_file, 'r', encoding='utf-8') as fd:
    xml_data = xmltodict.parse(fd.read())

In [ ]:
print(len(xml_data['pdml']['packet']))
print(type(xml_data['pdml']['packet']))
print(json.dumps(xml_data['pdml']['packet'][0]['proto'][5],indent=4))
print(type(xml_data['pdml']['packet'][0]['proto'][0]))

In [48]:
## checking the number of emails  using the server ready code "220"

count_mails = 0
sender = []
receiver = []
subject = []
mail_body = []

subject_str = ""
mail_body_str = ""

for packet in xml_data['pdml']['packet']:
  for proto in packet['proto']:
    if(proto['@name'] == 'smtp'):
      field = proto['field']
      for smtp in field:
        if(smtp['@name'] == 'smtp.response'):
          fields = smtp['field']
          for code in fields:
            if(code['@name'] == 'smtp.response.code' and code['@show'] == '220'):
              count_mails = count_mails + 1
              ongoing = 1
            elif(code['@name'] == 'smtp.response.code' and code['@show'] == '221'):
              ongoing = 0
              subject.append(subject_str)
              mail_body.append(mail_body_str)
              subject_str = ""
              mail_body_str = ""
              #print(json.dumps(code,indent=4))
        if(smtp['@name'] == 'smtp.command_line'):
          fields = smtp['field']
          #print(type(fields)) 
          #print(json.dumps(fields,indent=4)) 
          #print(len(fields))
          if(type(fields) is list):
            for code in fields:
              if(code['@name'] == 'smtp.req.parameter'):
                if(code['@show'].startswith("TO")):
                  receiver.append(code['@show'])
                elif(code['@show'].startswith("FROM")):
                  sender.append(code['@show'])
              #print(json.dumps(fields[i],indent=4))  
            
    if(proto['@name'] == 'imf'):
      field = proto['field']
      for smtp in field:
        if(smtp['@name'] == 'imf.subject'):
          subject_str = subject_str + smtp['@show']
        if(smtp['@name'] == 'imf.message_text'):
          mail_body_str = mail_body_str + smtp['field']['@show']
print("i) Number of emails sent in this entire conversation : " + str(count_mails))

print("senders:")
for i in sender:
  print(i)
print('\n')
print("receiver:")
for i in receiver:
  print(i)
print('\n')
for i in subject:
  print("subject start :")
  print(i)
  print("subject end :\n")

for i in mail_body:
  print("mail body start :")
  print(i)
  print("mail body end :\n")

i) Number of emails sent in this entire conversation : 2
senders:
FROM:<alice@iitkgp.ac.in> size=67
FROM:<bob@ucla.edu> size=64


receiver:
TO:<bob@ucla.edu>
TO:<alice@iitkgp.ac.in>


subject start :
Hi Bob
subject end :

subject start :
Hello Alice,
subject end :

mail body start :
Let us meet for Dinner at 8PM in Paris Cafe.  
mail body end :

mail body start :
Yes, sure. But please move it to 9.  
mail body end :

